В задании вам понадобится собрать генеративную модлель для языка

---

# LSTM (7 баллов)

В данной части нужно реализовать модель с ипользованием LSTM

In [1]:
!pip install --quiet sentencepiece datasets transformers


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import random
import torch 
import numpy as np
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
import os

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [ ]:
# Добавьте код для подготовки данных

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

Ниже вам нужно реализовать модель, которая по началу последовательности предсказывает следующий токен.
*   Модель получает на вход последовательность токенов, прогоняет её через LSTM и выдает вероятности следующего токена.  
*   Используйте LSTM из pytorch
*   Не забудьте про `batch_first`


In [ ]:
class RnnGenerator(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim, padding_idx=pad_token_idx)
        self.lstm = torch.nn.LSTM(embed_dim, embed_dim, num_layers=num_layers, batch_first=True)
        self.fc = torch.nn.Linear(embed_dim, vocab_size)


    #x: (batch_size, seq_len)
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        logits = self.fc(lstm_out)
        return logits


In [ ]:
tokenizer.vocab_size

8000

In [ ]:
vocab_size = tokenizer.vocab_size

model = RnnGenerator(vocab_size, 512, 2)

In [ ]:
batch = next(iter(train_dataloader))
model(batch[:, :-1]).shape

torch.Size([16, 256, 8000])

In [ ]:
_ = model.to(device)

In [ ]:
learning_rate = 0.0005

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(ignore_index = pad_token_idx)

Реализуйте обучение модели:
*  Не забудьте сдвинуть src и trg относительно друг друга.
*  Не забудьте про `clip_grad_norm_`
*  Данных очень много, для отладки лучше проходить только часть данных иначе этоха будет очень длинной

In [ ]:
def train_epoch(model, callback):
    model.train()
    losses = []
    n_inter = 0

    for batch in tqdm(train_dataloader):
        src = batch[:, :-1].to(device)  # Входная последовательность
        trg = batch[:, 1:].to(device)   # Последовательность целей (сдвинутая)
        
        optimizer.zero_grad()  # Обнуляем градиенты

        # Прогоняем через модель
        logits = model(src)

        # Вычисляем функцию потерь
        loss = criterion(logits.transpose(1, 2), trg)
        loss.backward()  # Обратное распространение ошибки

        # Клиппинг градиентов
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()  # Обновляем параметры
        losses.append(loss.item())

        n_inter += 1
        if n_inter % 500 == 0:
            callback(np.mean(losses))
            losses = []

In [ ]:
def eval_model(model):
    model.eval()
    losses = []

    for batch in test_dataloader:
        src = batch[:, :-1].to(device)
        trg = batch[:, 1:].to(device)
        
        with torch.no_grad():  
            logits = model(src)
            loss = criterion(logits.transpose(1, 2), trg)
        losses.append(loss.item())

    return np.mean(losses) 

In [ ]:
eval_model(model)

8.988071646009173

Получите `loss < 5.0` на тестовой выборке. 

Если модель обучается слишком быстро до значений <1.0 вы что-то напутали с данными.

In [ ]:
def callback(train_loss):
    eval_loss = eval_model(model)
    model.train()
    print(f'Epoch: {epoch+1:02} | train_loss = {train_loss:.5f}, eval_loss = {eval_loss:.5f}')

for epoch in trange(5):
    train_loss = train_epoch(model, callback)

In [ ]:
# Параметры модели
vocab_size = tokenizer.vocab_size
embed_dim = 512
num_layers = 2

# Инициализация модели, оптимизатора и функции потерь
model = RnnGenerator(vocab_size, embed_dim, num_layers).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
criterion = torch.nn.CrossEntropyLoss(ignore_index=pad_token_idx)

# Тренировка модели
for epoch in trange(5):
    train_epoch(model, callback)

# Генерация текста (5 баллов) 

Реализуйте функцию, которая продолжает текст.
1.   Переведите строчку в токены
2.   Реализуйте код который предсказывает вероятность следующей буквы
3.   Семплируйте следующую букву
4.   Повторяйте пункты 2-3 в цикле
5.   Преобразуйте токены в строчку



In [ ]:
def continues_sentence(sentence, model, max_len = 30):
    model.eval()    
    tokens = list(encode(sentence.lower()))
    for _ in range(max_len):
        input_ids = torch.tensor([tokens], dtype=torch.long).to(device)
        with torch.no_grad():
            logits = model(input_ids)
        next_token_logits = logits[:, -1, :]
        probabilities = torch.nn.functional.softmax(next_token_logits, dim=-1)
        next_token = torch.multinomial(probabilities, num_samples=1).item()
        tokens.append(next_token)
        if next_token == sep_token_idx:
            break    
    
    return decoder.decode(tokenizer.convert_ids_to_tokens(tokens))

In [ ]:
continues_sentence("Я помню чудное мгновенье", model)

'я помню чудное мгновенье, в руки напомнились! маркизов усмехнулся и даму до чаю : не один этого дома нет, даже заревывал восм приятелю'

In [ ]:
continues_sentence("Мой дядя самых честных правил,", model)

'мои дядя самых честных правил, позвольте...... между тем я имеющии бедныи лист на руках лета... всю тем дрожка, он расхо'

In [ ]:
continues_sentence("Четыре года потратил Деонардо на", model)

'четыре года потратил деонардо на тот день вечер, и первыи былев сам выскочил из мещан и кстати разговился им с деньгами. бецкая женщина'

In [ ]:
continues_sentence("Если сила плохих людей в том, что они вместе, то хорошим людям, чтобы стать силой, надо", model)

'если сила плохих людеи в том, что они вместе, то хорошим людям, чтобы стать силои, надо показывать любо, у нашеи обязанности и странности отречения человека ; в числе его, оно проитис налевые, - в том'